In [1]:
from hide_cell import toggle_code as hide_cell
# Pip install necessary package
!pip install -U --quiet  huggingface_hub
%pip install --upgrade --quiet  pgvector
%pip install --upgrade --quiet  langchain-openai
%pip install --upgrade --quiet  psycopg2-binary
%pip install --upgrade --quiet  tiktoken

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

In [3]:
import torch, os, numpy as np, pandas as pd
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, Trainer, TrainingArguments

In [4]:
from huggingface_hub import login
import os
token = os.getenv("HUGGINGFACEHUB_API_TOKEN") or os.getenv("HF_TOKEN") or os.getenv("HF_API_KEY")
if not token:
    raise RuntimeError("Set HUGGINGFACEHUB_API_TOKEN or HF_TOKEN before running.")
login(token=token)


In [5]:
dataset = load_dataset("squad")
print("Dataset loaded...")

answers_train, answers_valid = np.array([]), np.array([])

questions_train = np.array(dataset["train"]["question"])
questions_valid = np.array(dataset["validation"]["question"])

print(type(questions_train))
print(questions_train)

Dataset loaded...
<class 'numpy.ndarray'>
['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'
 'What is in front of the Notre Dame Main Building?'
 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?'
 ... 'With what Belorussian city does Kathmandu have a relationship?'
 'In what year did Kathmandu create its initial international relationship?'
 'What is KMC an initialism of?']


In [6]:
#x = pd.DataFrame(dataset["train"]["answers"])["text"]
type(x)

NameError: name 'x' is not defined

In [7]:
answers_train = pd.DataFrame(dataset["train"]["answers"])["text"]
answers_valid = pd.DataFrame(dataset["validation"]["answers"])["text"]
answers_train = np.array(answers_train)
answers_valid = np.array(answers_valid)

#print("Saving files...")

#np.save("data/questions_train.npy", questions_train)
#np.save("data/questions_valid.npy", questions_valid)
#np.save("data/answers_train.npy", answers_train)
#np.save("data/answers_valid.npy", answers_valid)

#print(len(answers), type(answers), answers[30])
#print(len(questions), type(questions), questions[30])

In [13]:
answers_train[:10]
answers_valid[:10]

array([list(['Denver Broncos', 'Denver Broncos', 'Denver Broncos']),
       list(['Carolina Panthers', 'Carolina Panthers', 'Carolina Panthers']),
       list(['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."]),
       list(['Denver Broncos', 'Denver Broncos', 'Denver Broncos']),
       list(['gold', 'gold', 'gold']),
       list(['"golden anniversary"', 'gold-themed', '"golden anniversary']),
       list(['February 7, 2016', 'February 7', 'February 7, 2016']),
       list(['American Football Conference', 'American Football Conference', 'American Football Conference']),
       list(['"golden anniversary"', 'gold-themed', 'gold']),
       list(['American Football Conference', 'American Football Conference', 'American Football Conference'])],
      dtype=object)

In [9]:
titles = np.array(dataset["train"]["title"])
articles = np.array(dataset["train"]["context"])
articles = pd.DataFrame(data=[titles, articles]).T.drop_duplicates()
articles.columns = ["Titles", "Content"]
articles = articles.reset_index(drop=True)
#articles.to_csv("articles.csv", sep='|', index=False)

In [10]:
articles = pd.read_csv("articles.csv", sep='|', )
print(articles.shape)
articles.head()

(14153, 2)


,Titles,Content
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha..."
1,University_of_Notre_Dame,"As at most other universities, Notre Dame's st..."
2,University_of_Notre_Dame,The university is the major seat of the Congre...
3,University_of_Notre_Dame,The College of Engineering was established in ...
4,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...


In [70]:
content = articles["Content"].apply(len)
print(content.min(), content.max())

151 3706


In [ ]:
sentences = list(articles["Content"])

In [98]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5')
model.eval()

print("Started tokenizing...")
all_sentence_embeddings = torch.randn(0)

# Tokenize sentences
for i in range(187, articles.shape[0] // 100 + 1):
    #print(i)
    encoded_input = tokenizer(sentences[i*100:min(i*100+100, articles.shape[0])], padding=True, truncation=True, return_tensors='pt')
    # for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
    # encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')
    
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
        # Perform pooling. In this case, cls pooling.
        sentence_embeddings = model_output[0][:, 0]
    # normalize embeddings
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
    all_sentence_embeddings = torch.cat((all_sentence_embeddings, sentence_embeddings), dim=0)
    
    if i % 10 == 0: print("... +1000 embeddings done")
    #print("Sentence embeddings:", sentence_embeddings)
    #print(type(sentence_embeddings))

Started tokenizing...


In [170]:
all_sentence_embeddings.size()
#articles.shape[0] // 100 + 1

torch.Size([282, 384])

In [103]:
all_sentence_embeddings[100]

tensor([-3.9940e-03,  1.0652e-01,  5.3358e-03,  5.3919e-02,  7.5591e-03,
        -4.0923e-02,  4.7185e-03, -3.0655e-02, -4.4956e-02,  1.1459e-02,
         3.8129e-02, -6.8489e-02,  2.5115e-02,  1.0912e-02, -1.6136e-02,
         4.8968e-02, -7.3896e-02, -2.3109e-02,  5.1400e-02, -1.3878e-02,
        -6.2728e-02, -1.7276e-02,  3.0784e-02, -3.7519e-03,  5.1968e-03,
         2.5703e-02, -1.7471e-02, -3.0858e-02, -4.6582e-03, -1.2557e-01,
        -2.9843e-02,  6.9997e-03, -1.9401e-02, -1.5090e-02, -4.3128e-02,
         2.7862e-02,  2.4029e-02,  2.4787e-04, -8.1142e-03,  6.7506e-02,
        -3.4796e-02,  5.2754e-02,  5.0778e-02, -1.2876e-02, -6.5529e-02,
         4.7585e-02, -2.1723e-03, -4.2348e-03, -2.5838e-02, -3.0642e-02,
        -2.0854e-02, -3.3128e-02, -3.7983e-02,  4.0164e-02, -5.4660e-03,
        -2.6455e-02,  6.3354e-02,  7.9515e-04,  2.3316e-02, -9.2074e-02,
         5.8536e-02, -1.0653e-02, -1.4199e-01,  2.7099e-02,  1.1381e-02,
         4.1571e-02,  1.6465e-03, -3.6719e-02, -7.0

In [97]:
all_sentence_embeddings = torch.randn(0)
all_sentence_embeddings
sentence_embeddings = torch.randn(3,2)
all_sentence_embeddings = torch.cat((all_sentence_embeddings, sentence_embeddings), dim=0)
all_sentence_embeddings = torch.cat((all_sentence_embeddings, sentence_embeddings), dim=0)
all_sentence_embeddings

tensor([[-0.7611,  0.5862],
        [-0.0470, -0.5591],
        [-0.3606, -0.7509],
        [-0.7611,  0.5862],
        [-0.0470, -0.5591],
        [-0.3606, -0.7509]])

In [ ]:
torch.cat((all_sentence_embeddings, sentence_embeddings), dim=0)

In [119]:
dataset["train"]["question"][4999:50]

'What do greenhouses do with solar energy?'

In [132]:
np.unique(np.array(dataset["train"]["title"][:5102]))

array(['2008_Sichuan_earthquake', 'Antibiotics', 'Beyoncé',
       'Frédéric_Chopin', 'Genocide', 'IPod', 'Montana', 'New_York_City',
       'Sino-Tibetan_relations_during_the_Ming_dynasty', 'Solar_energy',
       'Spectre_(2015_film)', 'The_Legend_of_Zelda:_Twilight_Princess',
       'To_Kill_a_Mockingbird', 'University_of_Notre_Dame'], dtype='<U46')

In [134]:
np.unique(np.array(dataset["validation"]["title"]))

array(['1973_oil_crisis', 'Amazon_rainforest',
       'American_Broadcasting_Company', 'Apollo_program', 'Black_Death',
       'Chloroplast', 'Civil_disobedience',
       'Computational_complexity_theory', 'Construction', 'Ctenophora',
       'Doctor_Who', 'Economic_inequality', 'European_Union_law', 'Force',
       'French_and_Indian_War', 'Fresno,_California', 'Genghis_Khan',
       'Geology', 'Harvard_University', 'Huguenot', 'Immune_system',
       'Imperialism', 'Intergovernmental_Panel_on_Climate_Change',
       'Islamism', 'Jacksonville,_Florida', 'Kenya', 'Martin_Luther',
       'Newcastle_upon_Tyne', 'Nikola_Tesla', 'Normans', 'Oxygen',
       'Packet_switching', 'Pharmacy', 'Prime_number', 'Private_school',
       'Rhine', 'Scottish_Parliament', 'Sky_(United_Kingdom)',
       'Southern_California', 'Steam_engine', 'Super_Bowl_50', 'Teacher',
       'United_Methodist_Church', 'University_of_Chicago',
       'Victoria_(Australia)', 'Victoria_and_Albert_Museum', 'Warsaw',
      

In [136]:
len(np.array(dataset["validation"]["context"]))
np

10570

In [150]:
num_of_qna = len(np.array(dataset["validation"]["context"]))
np.random.seed(90)
test_idx = np.random.choice(num_of_qna, size=num_of_qna//5, replace=False)

In [151]:
questions_test = questions_valid[test_idx]
answers_test = answers_valid[test_idx]

In [152]:
train_idx = np.setdiff1d(np.arange(num_of_qna), test_idx)
questions_train = questions_valid[train_idx]
answers_train = answers_valid[train_idx]

In [155]:
len(train_idx)

8456

In [156]:
len(test_idx)

2114

In [157]:
np.save("data/questions_train.npy", questions_train)
np.save("data/questions_test.npy", questions_test)
np.save("data/answers_train.npy", answers_train)
np.save("data/answers_test.npy", answers_test)

In [167]:
import numpy as np

In [171]:
a_test = np.load("data/answers_test.npy", allow_pickle=True)
a_train = np.load("data/answers_train.npy", allow_pickle=True)
q_test = np.load("data/questions_test.npy", allow_pickle=True)
q_train = np.load("data/questions_train.npy", allow_pickle=True)


TypeError: __main__.<lambda>() got multiple values for keyword argument 'allow_pickle'

In [176]:
titles = np.array(dataset["validation"]["title"])
articles = np.array(dataset["validation"]["context"])
articles = pd.DataFrame(data=[titles, articles]).T.drop_duplicates()
articles.columns = ["Titles", "Content"]
articles = articles.reset_index(drop=True)
articles.to_csv("data/articles.csv", sep='|', index=False)